# A simple L-shaped example

In [1]:
import calliope
import pandas as pd
import pyomo.kernel as pmo

## Setup energy system models using Calliope

In [ ]:
config = "national_scale/model.yaml"
first_hour = "2005-01-01T00:00"

subsets = [
    ("2005-01-01", "2005-01-07"),
    ("2005-01-08", "2005-01-14"),
    ("2005-01-15", "2005-01-21"),
]

N = len(subsets)

model = {
    "main": {"calliope": calliope.Model(config, override_dict={"config.init.time_subset": [first_hour, first_hour]})},
    "subs": [{"calliope": calliope.Model(config, override_dict={"config.init.time_subset": s})} for s in subsets],
}

for i, m in enumerate([model["main"]] + model["subs"]):
    m["calliope"].build()
    m["T"] = len(m["calliope"].inputs.timesteps)
    m["backend"] = m["calliope"].backend
    m["pyomo"] = m["backend"]._instance
    m["vars"] = m["backend"].variables.to_dataframe()
    ts = m["vars"].index.get_level_values("timesteps")
    m["vars"] = m["vars"].loc[ts == ts[0]]
    m["vars"].index = m["vars"].index.droplevel([2, 3])

## Prepare linking variables

In [3]:
link_var_names = ["flow_cap", "storage_cap", "link_flow_cap", "area_use"]
link_var_names = [vn for vn in model["main"]["vars"].columns if vn in link_var_names]
link_vars = [
    (name, i) for name in link_var_names for (i, el) in model["main"]["vars"][name].items() if not pd.isnull(el)
    ]

## Modify main-model

In [4]:
# Deactivate operational variables.
for name in model["main"]["vars"].columns:
    if name in link_var_names:
        continue
    for var in model["main"]["vars"][name].dropna().values:
        var.deactivate()

# Prepare cut constraints and approximation variable (theta).
model["main"]["pyomo"].c_cuts = pmo.constraint_list()
model["main"]["pyomo"].v_theta = pmo.variable_list()
for i in range(len(model["subs"])):
    model["main"]["pyomo"].v_theta.append(pmo.variable(domain=pmo.NonNegativeReals))

# Prepare objective.
total_invest_cost = model["main"]["backend"].get_global_expression("cost_investment_annualised").sum().item()
model["main"]["backend"].objectives.get("min_cost_optimisation").item().deactivate()
model["main"]["pyomo"].e_cost = total_invest_cost * model["subs"][0]["T"] / model["main"]["T"]
model["main"]["pyomo"].obj = pmo.objective(model["main"]["pyomo"].e_cost + sum(model["main"]["pyomo"].v_theta) / N)

# Ensure that linking variables are bounded.
for name, idx in link_vars:
    x = model["main"]["vars"].loc[idx, name]
    x.lb = x.lb or 0.0
    x.ub = x.ub or 1e6

### Adding bounds to main-model variables

Try not executing this cell and see how this affects the convergence.

In [5]:
# Ensure that linking variables are bounded.
for name, idx in link_vars:
    x = model["main"]["vars"].loc[idx, name]
    x.lb = x.lb or 0.0
    x.ub = x.ub or 1e6

## Modify sub-model

In [6]:
for m in model["subs"]:
    # Fixing linked variables, including penalized slack variables (to ensure feasibility).
    m["pyomo"].p_link = pmo.parameter_dict()
    m["pyomo"].c_link = pmo.constraint_dict()
    m["pyomo"].v_slack_pos = pmo.variable_dict()
    m["pyomo"].v_slack_neg = pmo.variable_dict()

    for name, idx in link_vars:
        m["pyomo"].p_link[(name, idx)] = pmo.parameter(0.0)
        m["pyomo"].v_slack_pos[(name, idx)] = pmo.variable(domain=pmo.NonNegativeReals)
        m["pyomo"].v_slack_neg[(name, idx)] = pmo.variable(domain=pmo.NonNegativeReals)
        m["pyomo"].c_link[(name, idx)] = pmo.constraint(
            m["pyomo"].p_link[(name, idx)] + m["pyomo"].v_slack_pos[(name, idx)] - m["pyomo"].v_slack_neg[(name, idx)]
            == m["vars"].loc[idx, name]
        )

    # Prepare objective.
    m["backend"].objectives.get("min_cost_optimisation").item().deactivate()
    m["pyomo"].obj = pmo.objective(
        m["backend"].get_global_expression("cost_operation_variable").sum().item()
        + 1e5 * (sum(m["pyomo"].v_slack_pos.values()) + sum(m["pyomo"].v_slack_neg.values()))
    )

## Prepare optimization

In [10]:
# Optimizer for main model.
opt_m = pmo.SolverFactory("gurobi")
opt_m.options["Method"] = 2
opt_m.options["Crossover"] = 0

# Optimizer for sub model (make sure duals are extracted).
opt_s = pmo.SolverFactory("gurobi")
for m in model["subs"]:
    m["pyomo"].dual.activate()

# Track best bounds.
lb, ub = -1e100, +1e100

## Start iteration

In [ ]:
print("  iter  |  lb         |  ub         |  gap")
print("--------|-------------|-------------|-------------")
for iter in range(200):
    # Optimize main.
    ret = opt_m.solve(model["main"]["pyomo"])

    for m in model["subs"]:
        # Fix linking vars.
        for name, idx in link_vars:
            m["pyomo"].p_link[(name, idx)].value = model["main"]["vars"].loc[idx, name].value or 0.0

        # Optimize sub.
        ret = opt_s.solve(m["pyomo"])

    obj_val_sub = [pmo.value(m["pyomo"].obj) for m in model["subs"]]

    # Update bounds.
    lb = max(lb, pmo.value(model["main"]["pyomo"].obj.expr))
    ub = min(ub, pmo.value(model["main"]["pyomo"].e_cost) + sum(obj_val_sub) / N)

    # Update gap, check termination, and log.
    gap = abs((ub - lb) / ub)
    if (iter % 5) == 0:
        print(f"  {iter:4d}  |  {lb:.3e}  |  {ub:.3e}  |  {gap:.1e}")
    if gap < 1e-2:
        print(f"Reached gap of {gap:.1e} after {iter + 1} iterations, terminating.")
        break

    for i, m in enumerate(model["subs"]):
        # Build cut.
        cut_rhs = obj_val_sub[i]
        for name, idx in link_vars:
            x = model["main"]["vars"].loc[idx, name]
            dual = m["pyomo"].dual.get(m["pyomo"].c_link[(name, idx)])
            cut_rhs -= dual * (x - x.value)

        # Add cut.
        model["main"]["pyomo"].c_cuts.append(pmo.constraint(model["main"]["pyomo"].v_theta[i] >= cut_rhs))